In [1]:
import kagglehub, numpy as np, tensorflow as tf, keras
from keras.utils import audio_dataset_from_directory
from keras import layers

In [2]:
# Download latest version
path = kagglehub.dataset_download("mohammedalkooheji/guitar-notes-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\patma\.cache\kagglehub\datasets\mohammedalkooheji\guitar-notes-dataset\versions\3


In [ ]:
trainData = audio_dataset_from_directory(
    path+"/Guitar Dataset",
    validation_split = 0.2,
    subset = "training",
    seed = 42
)


validationData = audio_dataset_from_directory(
    path+"/Guitar Dataset",
    validation_split = 0.2,
    subset = "validation",
    seed = 42
)

test_ds = audio_dataset_from_directory(
    path + "/Guitar Dataset",
    shuffle=False
)

Found 1452 files belonging to 43 classes.
Using 1162 files for training.
Found 1452 files belonging to 43 classes.
Using 290 files for validation.
Found 1452 files belonging to 43 classes.


In [13]:
model = keras.Sequential([
    keras.layers.Input(shape=(88200, 1)),        # 2 seconds at 44.1 kHz
    keras.layers.Conv1D(32, 9, activation='relu', padding='same'),
    keras.layers.MaxPooling1D(4),
    keras.layers.Conv1D(64, 9, activation='relu', padding='same'),
    keras.layers.MaxPooling1D(4),
 
    keras.layers.Conv1D(128, 9, activation='relu', padding='same'),
    keras.layers.MaxPooling1D(4),
 
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(43, activation="softmax")
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 88200, 32)      │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 22050, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 22050, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 5512, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 5512, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 1378, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 43)             │         5,547 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,219 (383.67 KB)

 Trainable params: 98,219 (383.67 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
#
model.compile(
    # optimizer="adam",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [15]:
# 
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="noteClassifier.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

# 
history = model.fit(
    trainData,
    epochs=50,
    validation_data=validationData,
    callbacks=callbacks
)

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.0318 - loss: 3.7490 - val_accuracy: 0.0138 - val_loss: 3.7301
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.0344 - loss: 3.6544 - val_accuracy: 0.0103 - val_loss: 3.6113
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.0723 - loss: 3.4702 - val_accuracy: 0.0759 - val_loss: 3.2772
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.1231 - loss: 3.0157 - val_accuracy: 0.1138 - val_loss: 2.7783
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.2022 - loss: 2.6078 - val_accuracy: 0.1897 - val_loss: 2.5054
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.2823 - loss: 2.3290 - val_accuracy: 0.3069 - val_loss: 2.2427
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.3227 - loss: 2.1435 - val_accuracy: 0.3483 - val_loss: 2.1139
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.3623 - loss: 1.9934 - val_accuracy: 0.3000 - val_loss:

In [16]:
test_model = keras.models.load_model(
    "convnet_from_scratch.keras"
)
test_loss, test_acc = test_model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.3f}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 11s 242ms/step - accuracy: 0.4814 - loss: 1.6369
Test accuracy: 0.481
